<a href="https://colab.research.google.com/github/100477706/Proyecto2_Aprendizaje/blob/main/Clustering_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. IDENTIFICACIÓN DE ESTUDIANTES**

---

**Grupo de Trabajo:** 27

Gabriel José Rivera Amor - 100477706@alumnos.uc3m.es

Santiago José Díaz Rodríguez - 100479095@alumnos.uc3m.es

**Enlace Github:** https://github.com/100477706/Proyecto2_Aprendizaje.git

# **1. CARGA DE DATOS**

---

Mediante este apartado realizamos la carga de datos al Google Collab teniendo en cuenta que en cada ingreso a la aplicación se debe realizar la carga del dataset.

En este apartado se descomprime el archivo que se ha subido y se muestran los datos por pantalla.

In [ ]:
!ls

sample_data  semillas.csv


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("semillas.csv", sep=",") #subir el archivo de semillas
df.head()

,area,perimetro,compacidad,longitud,anchura,asimetria,surco,clase
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


Como estamos haciendo un proceso de clustering, es necesario "eliminar" de los datos cargados dicha columna, ya que nosotros estamos buscando etiquetar a la clase a la que pertencen cada uno de los elementos.

In [ ]:
# Columna a eliminar para el clustering
delete_cols = ['clase']

# Retiramos la columna con la función drop
df_c = df.drop(columns=delete_cols)

# Mostramos nuevamenete el dataset para verificar que se ha eliminado la columna
df_c.head()

,area,perimetro,compacidad,longitud,anchura,asimetria,surco
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175


# **2. PRINCIPAL COMPONENT ANALYSIS (PCA)**

---

El clustering es una técnica de aprendizaje no supervisado, la cual consiste en organizar un conjunto de datos en grupos, de tal forma que todos los elementos del grupo sean similares entre sí y los datos de grupos diferentes sean lo más distinto posible.

Por ello, primeramente es necesario elegir un escaldor para realizar el proceso de normalización de los datos presentes en el dataset. Sin embargo, los datos pueden tener muchas dimensiones, por lo que hacemos uso de PCA para poder reducir la dimensionalidad a dos dimensiones y poder graficarlos.

Para que podamos reducir efectivamente la dimensionalidad, queremos que pocas componentes, se pueda observar la mayor proporción de varianza posible, ya que representa la mayor cantidad de información o dispersión que tienen los datos.

Para ello, procedemos a visualizar con cada uno de los escaladores, la varianza obtenida y la acumulada mientras aumentamos el número de componentes. Es importante decir que solo gestionaremos PCA con dos componentes como se ha indicado en el enunciado.